# 循环神经网络的从零开始实现

在本节，我们将从零开始实现一个基于字符级循环神经网络的语言模型，并在周杰伦专辑歌词数据集上训练一个模型来进行歌词创作。首先，我们读取周杰伦专辑歌词数据集。

In [1]:
import d2lzh as d2l
import math
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time

In [2]:
(corpus_indices, char_to_idx, idx_to_char,
 vocab_size) = d2l.load_data_jay_lyrics()

## One-hot 向量

为了将词表示成向量来输入进神经网络，一个简单的办法是使用 one-hot 向量。假设词典中不同字符的数量为 $N$（即`vocab_size`），每个字符已经同一个从 0 到 $N-1$ 的连续整数值索引一一对应。如果一个字符的索引是整数 $i$, 那么我们创建一个全 0 的长为 $N$ 的向量，并将其位置为 $i$ 的元素设成 1。该向量就是对原字符的 one-hot 向量。下面分别展示了索引为 0 和 2 的 one-hot 向量，向量长度等于词典大小。

In [3]:
nd.one_hot(nd.array([0, 2]), vocab_size)


[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]]
<NDArray 2x1027 @cpu(0)>

我们每次采样的小批量的形状是（批量大小，时间步数）。下面的函数将这样的小批量变换成数个可以输入进网络的形状为（批量大小，词典大小）的矩阵，总数与时间步数相等。也就是说，时间步 $t$ 的输入 $\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$，其中 $n$ 为批量大小，$d$ 为输入个数，即 one-hot 向量长度（词典大小）。

In [4]:
def to_onehot(X, size):  # 本函数已保存在 d2lzh 包中方便以后使用。
    return [nd.one_hot(x, size) for x in X.T]

X = nd.arange(10).reshape((2, 5))
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, (2, 1027))

In [9]:
X,inputs,vocab_size

(
 [[ 0.  1.  2.  3.  4.]
  [ 5.  6.  7.  8.  9.]]
 <NDArray 2x5 @cpu(0)>, [
  [[ 1.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]
  <NDArray 2x1027 @cpu(0)>, 
  [[ 0.  1.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]
  <NDArray 2x1027 @cpu(0)>, 
  [[ 0.  0.  1. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]
  <NDArray 2x1027 @cpu(0)>, 
  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]
  <NDArray 2x1027 @cpu(0)>, 
  [[ 0.  0.  0. ...,  0.  0.  0.]
   [ 0.  0.  0. ...,  0.  0.  0.]]
  <NDArray 2x1027 @cpu(0)>], 1027)

## 初始化模型参数

接下来，我们初始化模型参数。隐藏单元个数 `num_hiddens`是一个超参数。

In [10]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()
print('will use', ctx)

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

    # 隐藏层参数。
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = nd.zeros(num_hiddens, ctx=ctx)
    # 输出层参数。
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    # 附上梯度。
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

will use gpu(0)


## 定义模型

我们根据循环神经网络的计算表达式实现该模型。首先定义`init_rnn_state`函数来返回初始化的隐藏状态。它返回由一个形状为（批量大小，隐藏单元个数）的值为 0 的 NDArray 组成的元组。使用元组是为了更方便处理隐藏状态含有多个 NDArray 的情况。

In [11]:
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

下面的`rnn`函数定义了在一个时间步里如何计算隐藏状态和输出。这里的激活函数使用了 tanh 函数。[“多层感知机”](../chapter_deep-learning-basics/mlp.md)一节中介绍过，当元素在实数域上均匀分布时，tanh 函数值的均值为 0。

In [12]:
def rnn(inputs, state, params):
    # inputs 和 outputs 皆为 num_steps 个形状为（batch_size，vocab_size）的矩阵。
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X, W_xh) + nd.dot(H, W_hh) + b_h)
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

做个简单的测试来观察输出结果的个数（时间步数），以及第一个时间步的输出层输出形状和隐藏状态形状。

In [13]:
state = init_rnn_state(X.shape[0], num_hiddens, ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, (2, 1027), (2, 256))

## 定义预测函数

以下函数基于前缀`prefix`（含有数个字符的字符串）来预测接下来的`num_chars`个字符。这个函数稍显复杂，其中我们将循环神经单元`rnn`设置成了函数参数，这样在后面小节介绍其他循环神经网络时能重复使用这个函数。

In [14]:
# 本函数已保存在 d2lzh 包中方便以后使用。
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入。
        X = to_onehot(nd.array([output[-1]], ctx=ctx), vocab_size)
        # 计算输出和更新隐藏状态。
        (Y, state) = rnn(X, state, params)
        # 下一个时间步的输入是 prefix 里的字符或者当前的最佳预测字符。
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

我们先测试一下`predict_rnn`函数。我们将根据前缀“分开”创作长度为 10 个字符（不考虑前缀长度）的一段歌词。因为模型参数为随机值，所以预测结果也是随机的。

In [15]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            ctx, idx_to_char, char_to_idx)

'分开啃忙书轮然爱学防拳猫'

## 裁剪梯度

循环神经网络中较容易出现梯度衰减或爆炸。我们会在本章后面的小节中解释这个原因。为了应对梯度爆炸，我们可以裁剪梯度（clip gradient）。假设我们把所有模型参数梯度的元素拼接成一个向量 $\boldsymbol{g}$，并设裁剪的阈值是 $\theta$。裁剪后的梯度

$$ \min\left(\frac{\theta}{\|\boldsymbol{g}\|}, 1\right)\boldsymbol{g}$$

的 $L_2$ 范数不超过 $\theta$。

In [16]:
# 本函数已保存在 d2lzh 包中方便以后使用。
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx)
    for param in params:
        norm += (param.grad ** 2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

## 困惑度

我们通常使用困惑度（perplexity）来评价语言模型的好坏。回忆一下[“Softmax 回归”](../chapter_deep-learning-basics/softmax-regression.md)一节中交叉熵损失函数的定义。困惑度是对交叉熵损失函数做指数运算后得到的值。特别地，

* 最佳情况下，模型总是把标签类别的概率预测为 1。此时困惑度为 1。
* 最坏情况下，模型总是把标签类别的概率预测为 0。此时困惑度为正无穷。
* 基线情况下，模型总是预测所有类别的概率都相同。此时困惑度为类别个数。

显然，任何一个有效模型的困惑度必须小于类别个数。在本例中，困惑度必须小于词典大小`vocab_size`。

## 定义模型训练函数

跟之前章节的模型训练函数相比，这里的模型训练函数有以下几点不同：

1. 使用困惑度（perplexity）评价模型。
2. 在迭代模型参数前裁剪梯度。
3. 对时序数据采用不同采样方法将导致隐藏状态初始化的不同。相关的讨论可参考[“语言模型数据集（周杰伦专辑歌词）”](lang-model-dataset.md)一节。

另外，考虑到后面将介绍的其它循环神经网络，为了更通用，这里的函数实现更长一些。

In [17]:
# 本函数已保存在 d2lzh 包中方便以后使用。
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, ctx, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在 epoch 开始时初始化隐藏状态。
            state = init_rnn_state(batch_size, num_hiddens, ctx)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态。
                state = init_rnn_state(batch_size, num_hiddens, ctx)
            else:  # 否则需要使用 detach 函数从计算图分离隐藏状态。
                for s in state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X, vocab_size)
                # outputs 有 num_steps 个形状为（batch_size，vocab_size）的矩阵。
                (outputs, state) = rnn(inputs, state, params)
                # 拼接之后形状为（num_steps * batch_size，vocab_size）。
                outputs = nd.concat(*outputs, dim=0)
                # Y 的形状是（batch_size，num_steps），转置后再变成长度为
                # batch * num_steps 的向量，这样跟输出的行一一对应。
                y = Y.T.reshape((-1,))
                # 使用交叉熵损失计算平均分类误差。
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, ctx)  # 裁剪梯度。
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均。
            l_sum += l.asscalar() * y.size
            n += y.size

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(
                    prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx))

## 训练模型并创作歌词

现在我们可以训练模型了。首先，设置模型超参数。我们将根据前缀“分开”和“不分开”分别创作长度为 50 个字符（不考虑前缀长度）的一段歌词。我们每过 50 个迭代周期便根据当前训练的模型创作一段歌词。

In [18]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 200, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

下面采用随机采样训练模型并创作歌词。

In [19]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 65.831525, time 0.82 sec
 - 分开 我想要这生 我不能 想你的让我 我的可头女人 我想要 你子我 我不要 我给我的可头 有知的美栈 我
 - 不分开 我想想你 你我的外 就人的可爱女 一直的兮斑 我不能 想你的让我 我的可头女人 我想要 你子我 我
epoch 100, perplexity 9.702426, time 0.93 sec
 - 分开 爱什么 干什么 一颗四颗 回成线 一片我抬起头 有话去对医 三思过人有 有在在怕  有在梦里 我已
 - 不分开吗 我爱你这 你知的让我 红的可以女义 每彻就没有你的 我想要的有你在样 什么兵器的片边 我都店小二
epoch 150, perplexity 2.839339, time 0.80 sec
 - 分开 娘子的美丽默 风所水我的手据著天 想反的可飞 让我茶始 相你怕不 我已手这节奏 后知不觉 你已经离
 - 不分开扫 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 200, perplexity 1.588897, time 0.84 sec
 - 分开 几姥心不妥 你唱过没难过 心伤透 娘子她人在江南等我 泪不休 语沉默 娘子她人在江南等我 泪不休 
 - 不分开吗 我叫你爸 你打我妈 这样对吗干嘛这样 何必让午它点子走 瞎 说底三口斯日长 或草在最旧始日难解的


接下来采用相邻采样训练模型并创作歌词。

In [14]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 60.365549, time 0.26 sec
 - 分开 我想要你爱 我不 我不 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开 你不要你不 我不 我不 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
epoch 100, perplexity 7.035743, time 0.26 sec
 - 分开 我说儿这样 我有往再想 我不要再想 我不 我不 我不能 爱情我的见你 有场去 娘沉默 一九四酒 再
 - 不分开 想要我 别怪我 说你了么面对我 甩开球我满腔的怒火 我想揍你已经很久 别想躲 说你眼睛看着我 别发
epoch 150, perplexity 2.104224, time 0.26 sec
 - 分开 我不儿这力护的愿像还人落我 泪姥再一个写慢慢你回边 我给你的爱写在西元前 深埋在美索不达米亚平原 
 - 不分开 你堡一口我打你  怎现当只汉堡么 我想你这样牵着我 手 你不了不听不 痛静我这样打到 但那个人  
epoch 200, perplexity 1.291586, time 0.26 sec
 - 分开 我后么 是果就那每有人亏我身无不声向   我有到其实我 不要你的睡小 后你想你飞友 瞎透都会 说你
 - 不分开 再你一口我打天处 躲我说那睡背多 难那杂草的泥土 不会骑扫把的胖女巫 用拉丁文念咒语啦啦呜 她养的


## 小结

* 我们可以应用基于字符级循环神经网络的语言模型来生成文本序列，例如创作歌词。
* 当训练循环神经网络时，为了应对梯度爆炸，我们可以裁剪梯度。
* 困惑度是对交叉熵损失函数做指数运算后得到的值。


## 练习

* 调调超参数，观察并分析对运行时间、困惑度以及创作歌词的结果造成的影响。
* 不裁剪梯度，运行本节代码。结果会怎样？
* 将`pred_period`变量设为 1，观察未充分训练的模型（困惑度高）是如何创作歌词的。你获得了什么启发？
* 将相邻采样改为不从计算图分离隐藏状态，运行时间有没有变化？
* 将本节中使用的激活函数替换成 ReLU，重复本节的实验。


## 扫码直达[讨论区](https://discuss.gluon.ai/t/topic/989)

![](../img/qr_rnn-scratch.svg)